<a href="https://colab.research.google.com/github/hmghaly/word_align/blob/main/walign_nlp_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Start

In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
cwd='/content/drive/MyDrive/tr_ar' #directory where we keep the data
if not os.path.exists(cwd): os.makedirs(cwd)  #if the directory doesn't exist, create it
os.chdir(cwd) #change current work directory to the directory where we keep the data
!pwd

Mounted at /content/drive
/content/drive/MyDrive/tr_ar


In [2]:
!rm -r code_utils #first delete the code_utils folder, then clone from the codebase (very crude till we figure out using github from colab)
!git clone https://github.com/hmghaly/word_align.git code_utils

Cloning into 'code_utils'...
remote: Enumerating objects: 3685, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 3685 (delta 80), reused 0 (delta 0), pack-reused 3554 (from 4)
Receiving objects: 100% (3685/3685), 29.49 MiB | 14.59 MiB/s, done.
Resolving deltas: 100% (2018/2018), done.


#Bert Experiments

In [ ]:
import transformers
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-multilingual-cased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
!pip install transformers

import torch
import transformers
import itertools
from code_utils import general
from code_utils.general import *
from code_utils.arabic_lib import *
from code_utils.align_utils import *
from code_utils.pandas_utils import *
from code_utils.web_lib import *
from itertools import groupby
from collections import Counter
ar_counter_dict=load_counts("ar_count.txt", tmp_count_dict={})
def cur_ar_tok_fn(text):
  cur_toks=tok(text)
  return tok_ar(cur_toks,ar_counter_dict)



model = transformers.BertModel.from_pretrained('bert-base-multilingual-cased')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [ ]:
from code_utils.general import *
from code_utils.align_utils import *

#align_layer=8
#threshold = 1e-3
model.eval()

def bert_walign(src_tokens,trg_tokens,align_layer=8,n_epochs=8,max_dist=2,max_span=6):
  bert_out0=get_bert_align_list(src_toks0,trg_toks0,align_layer=align_layer)
  aligned_path0=get_aligned_path(bert_out0,n_epochs=n_epochs,max_dist=max_dist,max_span=max_span)
  return aligned_path0

def get_bert_align_list(src_tokens,trg_tokens,align_layer=8):
  #token_src, token_tgt = [tokenizer.tokenize(word) for word in sent_src], [tokenizer.tokenize(word) for word in sent_tgt]
  token_src, token_tgt = [tokenizer.tokenize(word) for word in src_tokens], [tokenizer.tokenize(word) for word in trg_tokens]
  wid_src, wid_tgt = [tokenizer.convert_tokens_to_ids(x) for x in token_src], [tokenizer.convert_tokens_to_ids(x) for x in token_tgt]
  ids_src, ids_tgt = tokenizer.prepare_for_model(list(itertools.chain(*wid_src)), return_tensors='pt', model_max_length=tokenizer.model_max_length, truncation=True)['input_ids'], tokenizer.prepare_for_model(list(itertools.chain(*wid_tgt)), return_tensors='pt', truncation=True, model_max_length=tokenizer.model_max_length)['input_ids']
  sub2word_map_src = []
  for i, word_list in enumerate(token_src):
    sub2word_map_src += [i for x in word_list]
  sub2word_map_tgt = []
  for i, word_list in enumerate(token_tgt):
    sub2word_map_tgt += [i for x in word_list]

  out_src = model(ids_src.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]
  out_tgt = model(ids_tgt.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]

  dot_prod = torch.matmul(out_src, out_tgt.transpose(-1, -2))

  softmax_srctgt = torch.nn.Softmax(dim=-1)(dot_prod)
  softmax_tgtsrc = torch.nn.Softmax(dim=-2)(dot_prod)

  align_dict={}
  temp_list=[]
  for i in range(len(softmax_srctgt)):
    word_i=sub2word_map_src[i]
    #src_word=sent_src[word_i] #src_tokens
    src_word=src_tokens[word_i] #src_tokens
    cur_row=softmax_srctgt[i]
    combined_row=[]
    for j in range(len(cur_row)):
      #combined_row.append((j,softmax_srctgt[i][j].item(),softmax_tgtsrc[i][j].item()))
      word_j=sub2word_map_tgt[j]
      #trg_word=sent_tgt[word_j] #trg_tokens
      trg_word=trg_tokens[word_j] #trg_tokens
      ij_val=round(softmax_srctgt[i][j].item(),4)
      ji_val=round(softmax_tgtsrc[i][j].item(),4)
      avg_val=combined_val=(ij_val+ji_val)/2
      max_val=max(ij_val,ji_val)
      new_val=0.5*(avg_val+max_val)
      #combined_val=ji_val
      if combined_val==0: continue
      if src_word=="the": continue
      if len(src_word.strip("_"))<2: continue
      if len(trg_word.strip("_"))<2: continue
      combined_row.append((word_j,trg_word,combined_val))
      temp_list.append(((word_i,word_j,src_word,trg_word), combined_val))
  temp_list.sort()
  grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(temp_list,lambda x:x[0])]
  raw_align_list=[]
  common_count=100
  for key0,grp0 in grouped:
    avg_val=sum(grp0)/len(grp0)
    avg_max=0.5*(max(grp0)+avg_val)
    x0,y0,src_word0,trg_word0=key0
    src_span=(x0,x0)
    trg_span=(y0,y0)
    wt0=round(avg_max,4)
    raw_align_list.append((src_word0,trg_word0,[src_span],[trg_span],common_count,wt0))
  return raw_align_list



src_sent='What policies and practices can be implemented to ensure that underrepresented groups of persons with disabilities, such as women and girls, Indigenous Peoples and persons living in rural or remote areas, are adequately represented and included in disability rights advocacy and policymaking at the local, national, and international levels?'
trg_sent='* ما هي السياسات والممارسات التي يمكن تنفيذها لضمان تمثيل الفئات الممثلة تمثيلا ناقصا من الأشخاص ذوي الإعاقة، مثل النساء والفتيات والشعوب الأصلية والأشخاص الذين يعيشون في المناطق الريفية أو النائية، تمثيلا كافيا وإشراكهم في الدعوة إلى حقوق الأشخاص ذوي الإعاقة وصنع السياسات على المستويات المحلية والوطنية والدولية؟'
#src_sent='In conclusion, I can only reaffirm the assessment of the report of the Secretary-General (A/76/311) that achieving the sustainability of our seas and oceans demands urgent and intensive efforts on all fronts and at all levels to reverse the negative trends and patterns, including by developing marine science and technology, conserving marine biodiversity and marine resources, using them sustainably and protecting and preserving the marine environment.	'
#trg_sent='في الختام، لا يسعني سوى التأكيد على ما جاء في تقرير الأمين العام للأمم المتحدة (A/76/311) حول أن تحقيق استدامة البحار والمحيطات، سيتطلب بذل جهود عاجلة وأكبر على جميع الجهات والمستويات، لقلب مسار الاتجاهات وأنماط الاستخدام السلبي، بما في ذلك من خلال تطوير العلوم والتكنولوجيات البحرية، وحفظ التنوع البيولوجي البحر والموارد البحرية، واستخدامها استخداما مستداما وحماية البيئة البحرية والحفاظ عليها.'
src_toks0, trg_toks0 = tok(src_sent.lower()),cur_ar_tok_fn(trg_sent.lower()) #src_sent.strip().split(), trg_sent.strip().split()
#bert_out=get_bert_align_list(src_toks0,trg_toks0)
# for a in bert_out:
#   print(a)

#aligned_path0=get_aligned_path(bert_out,n_epochs=8,max_dist=2,max_span=6)
aligned_path1=bert_walign(src_toks0,trg_toks0)
phrases0=present_aligned(src_toks0,trg_toks0,aligned_path1)

for bo in phrases0:
  print(bo)
#0.5 (average + maximum)
#average 1, max 1 >> 1 #average 0.5, max 1 >> 0.75 # then take the maximum of each pair of coords

('can be implemented to ensure that underrepresented', 'يمكن تنفيذ ـها لـ ضمان تمثيل', ((4, 10), (7, 12)), 4.8744)
('can', 'يمكن', ((4, 4), (7, 7)), 1.0)
('implemented to ensure that underrepresented', 'تنفيذ ـها لـ ضمان تمثيل', ((6, 10), (8, 12)), 3.8744)
('implemented to', 'تنفيذ ـها لـ', ((6, 7), (8, 10)), 2.0)
('implemented', 'تنفيذ', ((6, 6), (8, 8)), 1.0)
('to', 'لـ', ((7, 7), (10, 10)), 1.0)
('ensure that underrepresented', 'ضمان تمثيل', ((8, 10), (11, 12)), 1.8744)
('ensure', 'ضمان', ((8, 8), (11, 11)), 1.0)
('underrepresented', 'تمثيل', ((10, 10), (12, 12)), 0.8744)
('persons living in rural or remote areas', 'ال_أشخاص الذين يعيشون في ال_مناطق ال_ريفية أو', ((26, 32), (30, 36)), 4.1664)
('persons', 'ال_أشخاص', ((26, 26), (30, 30)), 0.8114)
('living in rural or remote areas', 'يعيشون في ال_مناطق ال_ريفية أو', ((27, 32), (32, 36)), 3.355)
('living', 'يعيشون', ((27, 27), (32, 32)), 0.875)
('rural or remote areas', 'ال_مناطق ال_ريفية أو', ((29, 32), (34, 36)), 2.48)
('areas', 'ال_

# Word Align - Phrase Align

In [ ]:
from code_utils.general import *
from code_utils.arabic_lib import *
from itertools import groupby
from collections import Counter

#compiling data
cur_zipfile="UNv1.0.en-ar-combined.zip"
zipped_fname='UNv1.0.en-ar-combined.txt'

subword_vec_size=100
empty_vec=[0.]*subword_vec_size
vec_inc=0.00001

src_subword_vec_dict,trg_subword_vec_dict={},{}
for i0,a0 in enumerate(iter_zip(cur_zipfile,zipped_fname,from_loc=0)):
  if i0>10000: break
  vec_i=i0 % subword_vec_size
  loc0,line0=a0
  src0,trg0=line0.strip().split("\t")
  src_toks0=tok(src0.lower())
  trg0=clean_ar(trg0.lower())
  trg_toks0=tok(trg0)
  src_subwords,trg_subwords=[],[]
  for s_0 in src_toks0: src_subwords.extend(get_char_ngrams(s_0,max_size=4))
  for t_0 in trg_toks0: trg_subwords.extend(get_char_ngrams(t_0,max_size=4))

  # print(src_toks0)
  # print(trg_toks0)
  # print("src_subwords",len(src_subwords), src_subwords)
  # print("trg_subwords", len(trg_subwords), trg_subwords)
  for src_sw0 in list(set(src_subwords)):
    cur_vec0=src_subword_vec_dict.get(src_sw0,list(empty_vec))
    cur_vec0[vec_i]+=vec_inc
    src_subword_vec_dict[src_sw0]=cur_vec0

  for trg_sw0 in list(set(trg_subwords)):
    cur_vec0=trg_subword_vec_dict.get(trg_sw0,list(empty_vec))
    cur_vec0[vec_i]+=vec_inc
    trg_subword_vec_dict[trg_sw0]=cur_vec0




  #print("----")


src_items=list(src_subword_vec_dict.items())
trg_items=list(trg_subword_vec_dict.items())
for a in src_items[:100]: print(a)
print(len(src_items),len(trg_items))


('#sp', [8e-05, 7.000000000000001e-05, 6e-05, 7.000000000000001e-05, 2e-05, 4e-05, 1e-05, 5e-05, 5e-05, 2e-05, 9e-05, 6e-05, 6e-05, 5e-05, 4e-05, 5e-05, 8e-05, 3.0000000000000004e-05, 7.000000000000001e-05, 8e-05, 3.0000000000000004e-05, 2e-05, 8e-05, 3.0000000000000004e-05, 2e-05, 1e-05, 7.000000000000001e-05, 3.0000000000000004e-05, 5e-05, 5e-05, 5e-05, 1e-05, 1e-05, 3.0000000000000004e-05, 4e-05, 2e-05, 2e-05, 5e-05, 1e-05, 2e-05, 4e-05, 5e-05, 3.0000000000000004e-05, 5e-05, 2e-05, 5e-05, 5e-05, 8e-05, 7.000000000000001e-05, 4e-05, 5e-05, 1e-05, 3.0000000000000004e-05, 2e-05, 3.0000000000000004e-05, 1e-05, 5e-05, 4e-05, 6e-05, 1e-05, 1e-05, 7.000000000000001e-05, 8e-05, 4e-05, 4e-05, 4e-05, 5e-05, 7.000000000000001e-05, 3.0000000000000004e-05, 5e-05, 8e-05, 6e-05, 4e-05, 6e-05, 3.0000000000000004e-05, 6e-05, 3.0000000000000004e-05, 4e-05, 7.000000000000001e-05, 3.0000000000000004e-05, 8e-05, 5e-05, 2e-05, 4e-05, 9e-05, 4e-05, 6e-05, 3.0000000000000004e-05, 3.0000000000000004e-05, 3.

In [ ]:
from scipy import spatial
#cosine similarity between word vectors
def cos_sim(vector1,vector2):
  if len(vector1)==0 or len(vector2)==0: return 0
  result = 1 - spatial.distance.cosine(vector1, vector2)
  return result

vec0=src_subword_vec_dict["ly#"]
vec1=trg_subword_vec_dict["بشكل"]
cos_sim(vec0,vec1)

word0='particularly'
word1='سيما'
src_ngrams=get_char_ngrams(word0,min_size=2,max_size=4)
trg_ngrams=get_char_ngrams(word1,min_size=2,max_size=4)
all_pairs=[]
for a in src_ngrams:
  vec0=src_subword_vec_dict[a]
  for b in trg_ngrams:
    vec1=trg_subword_vec_dict[b]
    sim0=cos_sim(vec0,vec1)
    all_pairs.append((a,b,sim0))
    #print(a,b,)
all_pairs.sort(key=lambda x:-x[-1])

for a0 in all_pairs[:50]:
  print(a0)


('ti', 'ا#', 0.9943265625432239)
('y#', 'ا#', 0.9926678186954763)
('#p', 'ا#', 0.9923867494035006)
('#p', 'ما', 0.9919681049776851)
('ar', 'ا#', 0.9918035405234081)
('ar', 'ا#', 0.9918035405234081)
('ti', 'ما', 0.990945996327572)
('y#', 'ما', 0.9908605838145937)
('ar', 'ما', 0.9907405412394267)
('ar', 'ما', 0.9907405412394267)
('rt', 'ا#', 0.9885170672806446)
('ic', 'ما', 0.9880177239782886)
('ic', 'ا#', 0.9878409553951961)
('rt', 'ما', 0.987165526508547)
('ly', 'ا#', 0.9811040334416917)
('la', 'ا#', 0.9791322272707658)
('ly#', 'ا#', 0.9791271880411807)
('la', 'ما', 0.9777679178261446)
('ly', 'ما', 0.9767493587398762)
('ul', 'ا#', 0.9766685397624665)
('pa', 'ما', 0.975924457633735)
('pa', 'ا#', 0.9758579065765669)
('ly#', 'ما', 0.9750343394747377)
('ti', 'سي', 0.974843355180888)
('ic', 'ما#', 0.9741952828364416)
('#p', 'سي', 0.972549461036198)
('ti', 'ما#', 0.9723600694912382)
('ul', 'ما', 0.9722383312509738)
('y#', 'ما#', 0.9721507475464336)
('ar', 'ما#', 0.9720525422397355)
('ar', 'م

In [ ]:
from scipy import spatial
#cosine similarity between word vectors
def cos_sim(vector1,vector2):
  if len(vector1)==0 or len(vector2)==0: return 0
  result = 1 - spatial.distance.cosine(vector1, vector2)
  return result

vec0=src_subword_vec_dict["ly#"]
vec1=trg_subword_vec_dict["بشكل"]
cos_sim(vec0,vec1)

0.6229219816563832

In [ ]:
word0='particularly'
#word0='الاختصاص'
#word0="ولذلك"

def get_char_ngrams(word,min_size=2,max_size=5,padding="#"):
  all_char_ngrams=[]
  word=padding+word+padding
  len_word=len(word)
  for size0 in range(min_size,max_size+1):
    for i0 in range(len_word-size0+1):
      cur_chunk=word[i0:i0+size0]
      all_char_ngrams.append(cur_chunk)
  return all_char_ngrams

get_char_ngrams(word0)

#get_char_chunks(word0,max_chunk_size=5,exclude_inside_chunks=False)

['#p',
 'pa',
 'ar',
 'rt',
 'ti',
 'ic',
 'cu',
 'ul',
 'la',
 'ar',
 'rl',
 'ly',
 'y#',
 '#pa',
 'par',
 'art',
 'rti',
 'tic',
 'icu',
 'cul',
 'ula',
 'lar',
 'arl',
 'rly',
 'ly#',
 '#par',
 'part',
 'arti',
 'rtic',
 'ticu',
 'icul',
 'cula',
 'ular',
 'larl',
 'arly',
 'rly#',
 '#part',
 'parti',
 'artic',
 'rticu',
 'ticul',
 'icula',
 'cular',
 'ularl',
 'larly',
 'arly#']

# Arabic NLP and Parsing

In [ ]:
from code_utils.general import *
from code_utils.arabic_lib import *
from itertools import groupby
from collections import Counter

# import transformers
# tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-multilingual-cased')

#compiling data
cur_zipfile="UNv1.0.en-ar-combined.zip"
zipped_fname='UNv1.0.en-ar-combined.txt'

ar_counter_dict=load_counts("ar_count.txt", tmp_count_dict={})

def cur_ar_tok_fn(text):
  cur_toks=tok(text)
  return tok_ar(cur_toks,ar_counter_dict)

for i0,a0 in enumerate(iter_zip(cur_zipfile,zipped_fname,from_loc=0)):
  if i0>100: break
  loc0,line0=a0
  src0,trg0=line0.strip().split("\t")
  src0=re.sub("\d","5",src0)
  trg0=re.sub("\d","5",trg0)
  trg0=clean_ar(trg0)
  trg_toks0=tok(trg0)
  for ar_tk0 in trg_toks0:
    candidates=get_pre_suf_candidates(ar_tk0)
    sorted_candiates=sort_filter(candidates,ar_counter_dict)
    top_morph,wt=sorted_candiates[0]
    pre,stem,suf,label=top_morph
    print(ar_tk0,top_morph)
  print(trg_toks0)

In [ ]:
!pip install tiktoken

# Subword tokenization + alignment

In [12]:
#get basic token count
import time
from code_utils.general import *
#from collections import Counter
import numpy as np

#matrix to check coordinates of matching tokens, increment if both occur, decrement if one occurs and one doesn't
def create_offset_matrix(src_i_list,trg_i_list,matrix_n,matrix_m,inc=0.01):
  temp_offset_array=np.zeros((matrix_n,matrix_m))
  temp_offset_array[tuple(src_i_list),:]=-inc
  temp_offset_array[:,tuple(trg_i_list)]=-inc
  for temp_src_i in src_i_list:
    temp_offset_array[temp_src_i,tuple(trg_i_list)]=inc
  return temp_offset_array


cur_zipfile="UNv1.0.en-ar-combined.zip"
zipped_fname='UNv1.0.en-ar-combined.txt'

all_src_subwords,all_trg_subwords=[],[]

src_counter,trg_counter={},{}
for i0,a0 in enumerate(iter_zip(cur_zipfile,zipped_fname,from_loc=0)):
  if i0>20000: break
  loc0,line0=a0
  src0,trg0=line0.strip().split("\t")
  trg0=clean_ar(trg0)
  src_toks0=tok(src0.lower())
  trg_toks0=tok(trg0.lower())
  for s_0 in src_toks0:
     src_counter[s_0]=src_counter.get(s_0,0)+1
  for t_0 in trg_toks0:
     trg_counter[t_0]=trg_counter.get(t_0,0)+1

print("obtained token count info")
src_sw_model=subword(src_counter)
trg_sw_model=subword(trg_counter)
print("created subword tokenization models")

src_counter_items=list(src_counter.items())
src_counter_items.sort(key=lambda x:-x[1])

trg_counter_items=list(trg_counter.items())
trg_counter_items.sort(key=lambda x:-x[1])

#creating subword counters
src_sw_counter,trg_sw_counter={},{}
for a,b in src_counter_items:
  src_sw=src_sw_model.tok(a)
  for ssw0 in src_sw: src_sw_counter[ssw0]=src_sw_counter.get(ssw0,0)+1
for a,b in trg_counter_items:
  trg_sw=trg_sw_model.tok(a)
  for tsw0 in trg_sw: trg_sw_counter[tsw0]=trg_sw_counter.get(tsw0,0)+1

# src_sw_counter_items=list(src_sw_counter.items())
# trg_sw_counter_items=list(trg_sw_counter.items())

all_src_sw_list=sorted(list(src_sw_counter.keys()))
all_trg_sw_list=sorted(list(trg_sw_counter.keys()))

# for a,b in src_sw_counter_items[:20]: print(a,b)
# for a,b in trg_sw_counter_items[:20]: print(a,b)
print("src subword count:", len(all_src_sw_list),"trg subword count:", len(all_trg_sw_list))
print("created counts for unique subwords, for use in corr matrix")

#creating corr matrix

n_src,n_trg=len(all_src_sw_list),len(all_trg_sw_list)
correspondence_array=np.zeros((n_src,n_trg))

print("correspondence_array",correspondence_array.shape)

src_idx_dict=dict(iter([(w,v) for v,w in enumerate(all_src_sw_list)]))
trg_idx_dict=dict(iter([(w,v) for v,w in enumerate(all_trg_sw_list)]))

print("all_src_tok_list",len(all_src_sw_list),"all_trg_tok_dict",len(all_trg_sw_list))

# print(all_src_sw_list[:20])
# print(all_trg_sw_list[:20])
print("created corr matrix")


obtained token count info
created subword tokenization models
src subword count: 2525 trg subword count: 3688
created counts for unique subwords, for use in corr matrix
correspondence_array (2525, 3688)
all_src_tok_list 2525 all_trg_tok_dict 3688
['!#', '"#', '#', '#"_#', '#$', '#$1', '#$1,', '#$2', '#$2,', '#$3', '#$4', '#$4,', '#$5', "#'", '#(_#', '#*', '#*#', '#-#', '#.', '#0']
['#', '#"', '#"_#', "#'", "#'_#", '#)', '#)_#', '#*', '#*#', '#,', '#-#', '#.', '#0', '#00', '#000', '#03#', '#1', '#1#', '#10', '#10#']
created corr matrix


##Now updating subword corr

In [58]:
def tok_sw(tokens,sw_model):
  final_sw_list=[]
  for tk0 in tokens: final_sw_list.extend(sw_model.tok(tk0))
  return final_sw_list

correspondence_array=np.zeros((n_src,n_trg))
print("now updaing correspondences")
sw_pairs=[]
tokenized_pairs=[]
counter0=0
for i0,a0 in enumerate(iter_zip(cur_zipfile,zipped_fname,from_loc=0)):
  if i0>5000: break
  if i0%100==0: print(i0)
  loc0,line0=a0
  src0,trg0=line0.strip().split("\t")
  trg0=clean_ar(trg0)
  src_toks0=tok(src0.lower())
  trg_toks0=tok(trg0.lower())
  tokenized_pairs.append((src_toks0,trg_toks0))
  cur_src_sw_list0=tok_sw(src_toks0,src_sw_model)
  cur_trg_sw_list0=tok_sw(trg_toks0,trg_sw_model)
  cur_src_ids0,cur_trg_ids0=[],[]
  for ssw0 in cur_src_sw_list0: cur_src_ids0.append(src_idx_dict[ssw0])
  for tsw0 in cur_trg_sw_list0: cur_trg_ids0.append(trg_idx_dict[tsw0])
  sw_pairs.append((cur_src_sw_list0,cur_trg_sw_list0))

  cur_offset_matrix=create_offset_matrix(cur_src_ids0,cur_trg_ids0,n_src,n_trg,inc=0.001)
  # print(cur_offset_matrix)
  # print(cur_offset_matrix.sum())
  correspondence_array+=cur_offset_matrix

  # print(cur_src_sw_list0)
  # print(cur_src_ids0)

  # print(cur_trg_sw_list0)
  # print(cur_trg_ids0)


  # cur_src_idx=[src_idx_dict[v] for v in cur_src_sw_list0]
  # cur_trg_idx=[trg_idx_dict[v] for v in cur_trg_sw_list0]

  # cur_src_idx=[(v,src_idx_dict[v]) for v in cur_src_sw_list0]
  # cur_trg_idx=[(v,trg_idx_dict[v]) for v in cur_trg_sw_list0]
  # print(sorted(cur_src_idx))
  # print(sorted(cur_trg_idx))

  # cur_offset_matrix=create_offset_matrix(cur_src_idx,cur_trg_idx,n_src,n_trg,inc=0.01)
  # # print(cur_offset_matrix)
  # # print(cur_offset_matrix.sum())
  # correspondence_array+=cur_offset_matrix
  #print("------------")


now updaing correspondences
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000


In [93]:
i0=235
cur_tokenized_pair0=tokenized_pairs[i0]
src_toks0,trg_toks0=cur_tokenized_pair0
src_sw0,trg_sw0=[],[]
src_tok_sw_dict,trg_tok_sw_dict={},{}
for s_tk0 in src_toks0:
  cur_src_tok_sw_list0=src_sw_model.tok(s_tk0)
  src_sw0.extend(cur_src_tok_sw_list0)
  src_tok_sw_dict[s_tk0]=cur_src_tok_sw_list0
for t_tk0 in trg_toks0:
  cur_trg_tok_sw_list0=trg_sw_model.tok(t_tk0)
  trg_sw0.extend(cur_trg_tok_sw_list0)
  trg_tok_sw_dict[t_tk0]=cur_trg_tok_sw_list0

print(src_tok_sw_dict)
print(trg_tok_sw_dict)

#src_sw0,trg_sw0=cur_sw_pair0
src_sw_ids0=[src_idx_dict[ssw0] for ssw0 in list(set(src_sw0))]
trg_sw_ids0=[trg_idx_dict[tsw0] for tsw0 in list(set(trg_sw0))]

src_sw_ids0=[(ssw0,src_idx_dict[ssw0]) for ssw0 in src_sw0]
trg_sw_ids0=[(tsw0,trg_idx_dict[tsw0]) for tsw0 in trg_sw0]
# src_sw_ids0=list(set(src_sw_ids0))
# trg_sw_ids0=list(set(trg_sw_ids0))

corr_list=[]
sw_corr_dict={}
for ssw0,ssw_id0 in src_sw_ids0:
  #print(ssw0,ssw_id0)
  for tsw0,tsw_id0 in trg_sw_ids0:
    corr_val=correspondence_array[ssw_id0][tsw_id0]
    #print(ssw0,ssw_id0,tsw0,tsw_id0,corr_val)
    corr_list.append((ssw0,tsw0,float(corr_val)))
    sw_corr_dict[(ssw0,tsw0)]=float(corr_val)

for s_tk0,ssw_list0 in src_tok_sw_dict.items():
  #print(s_tk0,s_sw0)
  for t_tk0,tsw_list0 in trg_tok_sw_dict.items():
    #print(s_tk0,t_tk0)
    sw_corr_list=[]
    only_corr_vals=[]
    for ssw0 in ssw_list0:
      for tsw0 in tsw_list0:
        corr_val0=sw_corr_dict[(ssw0,tsw0)]
        sw_corr_list.append((ssw0,tsw0,round(corr_val0,3)))
        only_corr_vals.append(corr_val0)

    #print(t_tk0,t_sw0)
    max_corr_val=max(only_corr_vals)
    avg_corr=sum(only_corr_vals)/len(only_corr_vals)
    sw_corr_list.sort(key=lambda x:-x[-1])
    used_a,used_b=[],[]
    effective_wt_list=[]
    for a,b,w in sw_corr_list:
      if a in used_a or b in used_b: continue
      used_a.append(a)
      used_b.append(b)
      #print(a,b,w)
      effective_wt_list.append(w)
    # print("--")
    # print(effective_wt_list)
    effective_avg_corr=sum(effective_wt_list)/len(effective_wt_list)
    combined_max_avg=(effective_avg_corr+max_corr_val)/2
    #print(effective_avg_corr)
    #print((s_tk0,t_tk0),round(effective_avg_corr,3),round(max_corr_val,3),round(combined_max_avg,3))

    #print((s_tk0,t_tk0),round(combined_max_avg,3))
    if max_corr_val>0:
      print(">>>>", (s_tk0,t_tk0),round(max_corr_val,3))
    else: print((s_tk0,t_tk0),round(max_corr_val,3))


    #print(s_tk0,t_tk0,round(max_corr_val,3),round(avg_corr,3), sw_corr_list[:5])
    if max_corr_val<0: continue
    #print(s_tk0,t_tk0,round(max_corr_val,3))
    #print(round(max_corr_val,3),round(avg_corr,3))
  print("-------------")

{'66': ['#6', '6#'], '_.': ['#_.#'], 'the': ['#the', '#'], 'creation': ['#', 'crea', 'tion', '#'], 'of': ['#of#'], 'mechanisms': ['#me', 'chan', 'is', 'm', 's#'], 'to': ['#to#'], 'bring': ['#b', 'r', 'ing#'], 'defendants': ['#de', 'f', 'enda', 'nts#'], 'befort': ['#be', 'f', 'ort#'], 'a': ['#a#'], 'court': ['#cou', 'rt#'], 'presents': ['#pre', 'se', 'nts#'], 'many': ['#man', 'y#'], 'difficult': ['#di', 'ffic', 'ult#'], 'questions': ['#', 'ques', 'tion', 's#'], '_,': ['#_,#'], 'which': ['#whi', 'ch#'], 'must': ['#m', 'u', 'st#'], 'be': ['#be#'], 'resolved': ['#res', 'ol', 'v', 'ed#'], 'if': ['#if#'], 'is': ['#is#'], 'operate': ['#', 'oper', 'ate#'], 'effectively': ['#eff', 'ec', 'tive', 'ly#']}
{'66': ['#66#'], '-': ['#-#'], 'يثير': ['#ي', 'ث', 'ير#'], 'إنشاء': ['#إنش', 'اء#'], 'آليات': ['#آل', 'يات#'], 'لوضع': ['#ل', 'وضع#'], 'المتهمين': ['#الم', 'ته', 'مين#'], 'تحت': ['#تحت', '#'], 'تصرف': ['#ت', 'صرف#'], 'المحكمة': ['#الم', 'حك', 'مة#'], 'مسائل': ['#مس', 'ائل#'], 'صعبة': ['#ص', 'ع', 

In [57]:
i0=25
cur_sw_pair0=sw_pairs[i0]
src_sw0,trg_sw0=cur_sw_pair0
src_sw_ids0=[src_idx_dict[ssw0] for ssw0 in src_sw0]
trg_sw_ids0=[trg_idx_dict[tsw0] for tsw0 in trg_sw0]

src_sw_ids0=[(ssw0,src_idx_dict[ssw0]) for ssw0 in src_sw0]
trg_sw_ids0=[(tsw0,trg_idx_dict[tsw0]) for tsw0 in trg_sw0]
src_sw_ids0=list(set(src_sw_ids0))
trg_sw_ids0=list(set(trg_sw_ids0))

corr_list=[]
for ssw0,ssw_id0 in src_sw_ids0:
  #print(ssw0,ssw_id0)
  for tsw0,tsw_id0 in trg_sw_ids0:

    corr_val=correspondence_array[ssw_id0][tsw_id0]
    #print(ssw0,ssw_id0,tsw0,tsw_id0,corr_val)
    corr_list.append((ssw0,tsw0,float(corr_val)))
    #print(tsw0,tsw_id0)

#corr_list.sort(key=lambda x:-x[-1])
corr_list.sort(key=lambda x:(x[0], -x[-1]))

for a in corr_list:
  print(a)

print(src_sw0)
print(trg_sw0)
# basic_corr_dict={}
# # for src_sw0,src_sw_id0 in zip(src_sw0,src_sw_ids0):
# #   #print(src_sw0,src_sw_id0)

# #   for trg_sw0,trg_sw_id0 in zip(trg_sw0,trg_sw_ids0):
# #     print(src_sw0,src_sw_id0,trg_sw0,trg_sw_id0)
# #     corr_val=correspondence_array[src_sw_id0][trg_sw_id0]
# #     #print(src_sw0,trg_sw0,corr_val)

# for src_sw0,src_sw_id0 in zip(src_sw0,src_sw_ids0):
#   print(src_sw0,src_sw_id0)

# for trg_sw0,trg_sw_id0 in zip(trg_sw0,trg_sw_ids0):
#   print(trg_sw0,trg_sw_id0)


# print(src_sw0)
# print(src_sw_ids0)

# print(trg_sw0)
# print(trg_sw_ids0)

('#', '#', 2.8939999999997923)
('#', '#ال', 2.114999999999878)
('#', '#_.#', -0.006000000000000104)
('#', '#الم', -0.10200000000000012)
('#', '#_،#', -1.840999999999908)
('#', '#وال', -2.449999999999841)
('#', '#الع', -2.477999999999838)
('#', '#)_#', -2.4939999999998363)
('#', '#لل', -2.6129999999998232)
('#', '#_(#', -3.0269999999997776)
('#', '#_/_', -3.0649999999997735)
('#', 'د', -3.4629999999997296)
('#', 'امة#', -3.6719999999997066)
('#', '#1#', -3.884999999999683)
('#', 'ون#', -3.893999999999682)
('#', 'عية#', -3.91299999999968)
('#', 'مية#', -4.0899999999997005)
('#', 'جم', -4.099999999999704)
('#', 'ورة#', -4.213999999999742)
('#', 'سا', -4.235999999999749)
('#', 'بعة#', -4.292999999999768)
('#', '#_', -4.305999999999773)
('#', 'أربع', -4.310999999999774)
('#', '#10#', -4.351999999999788)
('#', '#47#', -4.3879999999998)
('#', 'رس', -4.390999999999801)
('#', 'رفق#', -4.404999999999806)
('#', '(_#', -4.411999999999808)
('#', '#a#', -4.438999999999817)
('#', ')_#', -4.4439999999

In [11]:
create_offset_matrix([0,1,3],[2,4],5,5,inc=0.01)

array([[-0.01, -0.01,  0.01, -0.01,  0.01],
       [-0.01, -0.01,  0.01, -0.01,  0.01],
       [ 0.  ,  0.  , -0.01,  0.  , -0.01],
       [-0.01, -0.01,  0.01, -0.01,  0.01],
       [ 0.  ,  0.  , -0.01,  0.  , -0.01]])

In [4]:
import time

src_sw_model=subword(src_counter)
trg_sw_model=subword(trg_counter)

word0="إنجازات"
word0="واستنتاجات"
word0="findings"
t0=time.time()
for _ in range(1000):
  sw_toks=src_sw_model.tok(word0)
  #sw_toks=trg_sw_model.tok(word0)
t1=time.time()
elapsed0=t1-t0
print(sw_toks, elapsed0)

['#fin', 'ding', 's#'] 0.039388418197631836


In [ ]:
src_counter_items=list(src_counter.items())
src_counter_items.sort(key=lambda x:-x[1])

trg_counter_items=list(trg_counter.items())
trg_counter_items.sort(key=lambda x:-x[1])


src_sw_counter,trg_sw_counter={},{}
for a,b in src_counter_items:
  src_sw=src_sw_model.tok(a)
  for ssw0 in src_sw: src_sw_counter[ssw0]=src_sw_counter.get(ssw0,0)+1
for a,b in trg_counter_items:
  trg_sw=trg_sw_model.tok(a)
  for tsw0 in trg_sw: trg_sw_counter[tsw0]=trg_sw_counter.get(tsw0,0)+1

src_sw_counter_items=list(src_sw_counter.items())
trg_sw_counter_items=list(trg_sw_counter.items())

for a,b in src_sw_counter_items[:20]: print(a,b)
for a,b in trg_sw_counter_items[:20]: print(a,b)
print(len(src_sw_counter_items),len(trg_sw_counter_items))

#the 31
# 2383
#of# 1
#_,# 1
#_.# 1
and# 37
#to# 1
#in# 1
#_-_ 1
#for 70
#(_# 1
#a# 1
#_/_ 1
#tha 7
t# 99
#_)# 1
#on# 1
#gen 24
e 212
ral# 38
#_،# 1
#في# 1
#_.# 1
#من# 1
#_/_ 1
# 3764
#-# 1
#)_# 1
إلى# 3
على# 6
#أن# 1
#_(# 1
#الم 1512
ت 638
حدة# 17
#الأ 311
مم# 11
#_(_ 1
#الت 551
ي# 830
2525 3688


In [ ]:
import numpy as np

def create_offset_matrix(src_i_list,trg_i_list,matrix_n,matrix_m,inc=0.01):
  temp_offset_array=np.zeros((matrix_n,matrix_m))
  temp_offset_array[tuple(src_i_list),:]=-inc
  temp_offset_array[:,tuple(trg_i_list)]=-inc
  for temp_src_i in src_i_list:
    temp_offset_array[temp_src_i,tuple(trg_i_list)]=inc
  return temp_offset_array


all_src_tok_list=sorted(list(src_sw_counter.keys()))
all_trg_tok_list=sorted(list(trg_sw_counter.keys()))

n_src,n_trg=len(all_src_tok_list),len(all_trg_tok_list)
correspondence_array=np.zeros((n_src,n_trg))

print("correspondence_array",correspondence_array.shape)

src_idx_dict=dict(iter([(w,v) for v,w in enumerate(all_src_tok_list)]))
trg_idx_dict=dict(iter([(w,v) for v,w in enumerate(all_trg_tok_list)]))

print("all_src_tok_list",len(all_src_tok_list),"all_trg_tok_dict",len(all_trg_tok_list))

print(all_src_tok_list[:20])
print(all_trg_tok_list[:20])

print("now updaing correspondences")
counter0=0
for i0,a0 in enumerate(iter_zip(cur_zipfile,zipped_fname,from_loc=0)):
  if i0>20: break
  loc0,line0=a0
  src0,trg0=line0.strip().split("\t")
  trg0=clean_ar(trg0)
  src_toks0=tok(src0.lower())
  trg_toks0=tok(trg0.lower())
  cur_src_sw_list0,cur_trg_sw_list0=[],[]
  for s_0 in src_toks0:
    sw0=src_sw_model.tok(s_0)
    print(s_0,sw0)
    cur_src_sw_list0.extend(sw0)
  for t_0 in trg_toks0: cur_trg_sw_list0.extend(trg_sw_model.tok(t_0))
  print(src0)
  print(trg0)
  print(cur_src_sw_list0)
  print(cur_trg_sw_list0)

  cur_src_idx=[src_idx_dict[v] for v in cur_src_sw_list0]
  cur_trg_idx=[trg_idx_dict[v] for v in cur_trg_sw_list0]

  cur_src_idx=[(v,src_idx_dict[v]) for v in cur_src_sw_list0]
  cur_trg_idx=[(v,trg_idx_dict[v]) for v in cur_trg_sw_list0]
  print(sorted(cur_src_idx))
  print(sorted(cur_trg_idx))

  # cur_offset_matrix=create_offset_matrix(cur_src_idx,cur_trg_idx,n_src,n_trg,inc=0.01)
  # # print(cur_offset_matrix)
  # # print(cur_offset_matrix.sum())
  # correspondence_array+=cur_offset_matrix
  print("------------")





# for src_toks0,trg_toks0 in zip(src_sent_list,trg_sent_list):
#   if counter0%50==0: print(counter0)
#   counter0+=1
#   # print(src_toks0,trg_toks0)
#   cur_src_idx=[src_idx_dict[v] for v in src_toks0]
#   cur_trg_idx=[trg_idx_dict[v] for v in trg_toks0]
#   cur_offset_matrix=create_offset_matrix(cur_src_idx,cur_trg_idx,n_src,n_trg,inc=0.01)
#   # print(sorted(cur_src_idx))
#   # print(sorted(cur_trg_idx))
#   # print(cur_offset_matrix)
#   # print(cur_offset_matrix.sum())
#   correspondence_array+=cur_offset_matrix



correspondence_array (2525, 3688)
all_src_tok_list 2525 all_trg_tok_dict 3688
['!#', '"#', '#', '#"_#', '#$', '#$1', '#$1,', '#$2', '#$2,', '#$3', '#$4', '#$4,', '#$5', "#'", '#(_#', '#*', '#*#', '#-#', '#.', '#0']
['#', '#"', '#"_#', "#'", "#'_#", '#)', '#)_#', '#*', '#*#', '#,', '#-#', '#.', '#0', '#00', '#000', '#03#', '#1', '#1#', '#10', '#10#']
now updaing correspondences
original ['#or', 'igi', 'nal#']
_: ['#_:#']
english ['#en', 'g', 'lish', '#']
_/_ ['#_/_', '#']
spanish ['#spa', 'n', 'ish#']
ORIGINAL: ENGLISH/SPANISH
ORIGINAL: ENGLISH/SPANISH
['#or', 'igi', 'nal#', '#_:#', '#en', 'g', 'lish', '#', '#_/_', '#', '#spa', 'n', 'ish#']
['#ori', 'gi', 'nal#', '#_:#', '#eng', 'l', 'ish#', '#_/_', '#', '#', 'span', 'ish#']
[('#', 2), ('#', 2), ('#_/_', 120), ('#_:#', 121), ('#en', 227), ('#or', 354), ('#spa', 406), ('g', 1269), ('igi', 1393), ('ish#', 1453), ('lish', 1570), ('n', 1672), ('nal#', 1678)]
[('#', 0), ('#', 0), ('#_/_', 133), ('#_:#', 134), ('#eng', 159), ('#ori', 189), ('

In [ ]:
src_counter_items=list(src_counter.items())
src_counter_items.sort(key=lambda x:-x[1])
trg_counter_items=list(trg_counter.items())
trg_counter_items.sort(key=lambda x:-x[1])

max_subword_len=5

src_n_gram_counter,trg_n_gram_counter={},{}
for a,b in src_counter_items:
  char_ngrams=get_char_ngrams(a,max_size=max_subword_len)
  for ng0 in char_ngrams: src_n_gram_counter[ng0]=src_n_gram_counter.get(ng0,0)+b

for a,b in trg_counter_items:
  char_ngrams=get_char_ngrams(a,max_size=max_subword_len)
  for ng0 in char_ngrams: trg_n_gram_counter[ng0]=trg_n_gram_counter.get(ng0,0)+b


print("n_words src/trg", len(src_counter_items),len(trg_counter_items))

src_n_gram_counter_items=list(src_n_gram_counter.items())
src_n_gram_counter_items.sort(key=lambda x:-x[1])

trg_n_gram_counter_items=list(trg_n_gram_counter.items())
trg_n_gram_counter_items.sort(key=lambda x:-x[1])


for a in src_n_gram_counter_items[:100]: print(a)
print(len(src_n_gram_counter_items))

for a in trg_n_gram_counter_items[:100]: print(a)
print(len(trg_n_gram_counter_items))

n_words src/trg 16044 32190
('e#', 79246)
('#t', 66166)
('#_', 59406)
('th', 55951)
('s#', 52484)
('#a', 51830)
('#th', 47827)
('he', 45543)
('on', 44717)
('the', 42500)
('n#', 40637)
('d#', 40624)
('#the', 40565)
('in', 38837)
('#o', 38581)
('he#', 38493)
('the#', 38249)
('#the#', 38248)
('ti', 36099)
('re', 35366)
('er', 33818)
('at', 33746)
('t#', 32061)
('an', 30877)
('io', 29813)
('en', 29164)
('#i', 28419)
('ion', 27516)
('te', 26504)
('of', 26120)
('f#', 26052)
('#of', 25776)
('es', 25698)
('#c', 25511)
('#s', 24661)
('or', 24485)
('nt', 24400)
('of#', 24339)
('#of#', 24306)
('r#', 23463)
('it', 23133)
('_#', 22778)
('#p', 22144)
('tio', 22081)
('tion', 22047)
('nd', 21578)
('y#', 21544)
('al', 21379)
('ed', 21308)
('l#', 21140)
('co', 21089)
('on#', 20886)
('ed#', 19130)
('_,', 18757)
('#_,', 18757)
(',#', 18744)
('_,#', 18744)
('#_,#', 18744)
('#r', 18664)
('se', 18195)
('ar', 18006)
('#f', 17618)
('#in', 17613)
('st', 17592)
('#co', 17405)
('.#', 17334)
('ra', 17249)
('_.', 1

In [ ]:
import time
class subword: #create a subword tokenization model based on the token counter dict and certain parameters
  def __init__(self,counter_dict,params={}) -> None:
    self.min_size=params.get("min_size",2)
    self.max_size=params.get("max_size",4)
    self.padding="#"
    self.n_gram_counter={}
    for a,b in counter_dict:
      char_ngrams=get_char_ngrams(a,max_size=self.max_size,min_size=self.min_size,padding=self.padding)
      for ng0 in char_ngrams: self.n_gram_counter[ng0]=self.n_gram_counter.get(ng0,0)+b
  def tok(self,word):
    padded=self.padding+word+self.padding
    ng_list0=get_char_ngrams(word0,max_size=self.max_size,min_size=self.min_size,padding=self.padding,include_span=True)
    list0=[]
    for ng0,span0 in ng_list0:
      count0=self.n_gram_counter.get(ng0,0)+1
      log_count0=math.log10(count0)
      log_len0=len(ng0)#*10
      wt0=log_count0*log_len0 #TODO - maybe experiment with different ways to determine weight

      list0.append((ng0, span0,wt0))
    list0.sort(key=lambda x:-x[-1])
    used_indexes=[]
    final_list=[]
    for li0 in list0:
      ng0, span0,wt0=li0
      x0,x1=span0
      cur_indexes=list(range(x0,x1))
      intersection=list(set(cur_indexes).intersection(set(used_indexes)))
      if intersection: continue
      used_indexes.extend(cur_indexes)
      final_list.append(li0)

    #TODO - improve filling of remaining indexes
    remaining_indexes=list(set(range(len(padded))).difference(set(used_indexes)))
    for ri0 in remaining_indexes:
      span0=(ri0,ri0+1)
      chunk0=padded[ri0]
      final_list.append((chunk0,span0,0))
    final_list.sort(key=lambda x:x[1])

    return [v[0] for v in final_list]


src_sw_model=subword(src_counter_items)
word0="screaming"
t0=time.time()
for _ in range(1000):
  sw_toks=src_sw_model.tok(word0)
t1=time.time()
elapsed0=t1-t0
print(sw_toks, elapsed0)

['#s', 'crea', 'm', 'ing#'] 0.05498862266540527


In [ ]:
import math
word0="firefighting"
ng_list0=get_char_ngrams(word0,max_size=max_subword_len,include_span=True)
ng_list0.sort(key=lambda x:x[1])
wt_dict,path_dict={},{}
list0=[]
for ng0,span0 in ng_list0:
  count0=src_n_gram_counter.get(ng0,0)+1
  log_count0=math.log10(count0)
  #log_len0=math.log10(len(ng0)+1)
  log_len0=len(ng0)#*10
  wt0=log_count0*log_len0
  x0,x1=span0
  prev_wt=wt_dict.get(x0,0)
  cur_wt=wt_dict.get(x1,0)
  new_combined_wt=prev_wt+wt0
  if new_combined_wt>cur_wt:
    #print(ng0,span0,"new_combined_wt",new_combined_wt)
    wt_dict[x1]=new_combined_wt
    path_dict[x1]=x0
  print(ng0, span0, count0,round(log_count0,2),"log_len",round(log_len0,2),"wt0",round(wt0,2))
  list0.append((ng0, span0,wt0))

padded0="#"+word0+"#"
print(padded0)
# last_i=len(padded0)#+2
# prev0=last_i #path_dict.get(last_i)
# while prev0!=None:
#   old_prev0=int(prev0)
#   prev0=path_dict.get(prev0)
#   if prev0!=None:
#     chunk0=padded0[prev0:old_prev0]
#     print([prev0,old_prev0],chunk0)
list0.sort(key=lambda x:-x[-1])
used_indexes=[]
final_list=[]
for li0 in list0:
  ng0, span0,wt0=li0
  x0,x1=span0
  cur_indexes=list(range(x0,x1))
  intersection=list(set(cur_indexes).intersection(set(used_indexes)))
  if intersection: continue
  # if intersection:
  #   print(intersection)
  print(li0,cur_indexes,"intersection",intersection)
  used_indexes.extend(cur_indexes)
  final_list.append(li0)
print("-------")

remaining_indexes=list(set(range(len(padded0))).difference(set(used_indexes)))
for ri0 in remaining_indexes:
  span0=(ri0,ri0+1)
  chunk0=padded0[ri0]
  final_list.append((chunk0,span0,0))
final_list.sort(key=lambda x:x[1])
for a in final_list: print(a)
print(remaining_indexes)

#f (0, 2) 17619 4.25 log_len 2 wt0 8.49
#fi (0, 3) 2034 3.31 log_len 3 wt0 9.93
#fir (0, 4) 372 2.57 log_len 4 wt0 10.28
#fire (0, 5) 55 1.74 log_len 5 wt0 8.7
fi (1, 3) 4901 3.69 log_len 2 wt0 7.38
fir (1, 4) 520 2.72 log_len 3 wt0 8.15
fire (1, 5) 55 1.74 log_len 4 wt0 6.96
firef (1, 6) 1 0.0 log_len 5 wt0 0.0
ir (2, 4) 4571 3.66 log_len 2 wt0 7.32
ire (2, 5) 1578 3.2 log_len 3 wt0 9.59
iref (2, 6) 44 1.64 log_len 4 wt0 6.57
irefi (2, 7) 1 0.0 log_len 5 wt0 0.0
re (3, 5) 35367 4.55 log_len 2 wt0 9.1
ref (3, 6) 1402 3.15 log_len 3 wt0 9.44
refi (3, 7) 13 1.11 log_len 4 wt0 4.46
refig (3, 8) 1 0.0 log_len 5 wt0 0.0
ef (4, 6) 3118 3.49 log_len 2 wt0 6.99
efi (4, 7) 208 2.32 log_len 3 wt0 6.95
efig (4, 8) 1 0.0 log_len 4 wt0 0.0
efigh (4, 9) 1 0.0 log_len 5 wt0 0.0
fi (5, 7) 4901 3.69 log_len 2 wt0 7.38
fig (5, 8) 54 1.73 log_len 3 wt0 5.2
figh (5, 9) 27 1.43 log_len 4 wt0 5.73
fight (5, 10) 27 1.43 log_len 5 wt0 7.16
ig (6, 8) 3530 3.55 log_len 2 wt0 7.1
igh (6, 9) 2111 3.32 log_len 3 w

In [ ]:
enum_list=list(enumerate(final_list))
for z0,z1 in zip(enum_list,enum_list[1:]):
  i0,item0=z0
  i1,item1=z1
  span0,span1=item0[1],item1[1]
  ng0,ng1=item0[0],item1[0]
  print(span0,span1)

(0, 2) (2, 4)
(2, 4) (5, 7)
(5, 7) (8, 12)
(8, 12) (12, 16)


In [ ]:
# for a,b in path_dict.items():
#   print(a,b)

padded0="#"+word0+"#"
print(padded0)
last_i=len(padded0)#+2
prev0=last_i #path_dict.get(last_i)
while prev0!=None:
  old_prev0=int(prev0)
  prev0=path_dict.get(prev0)
  if prev0!=None:
    chunk0=padded0[prev0:old_prev0]
    print([prev0,old_prev0],chunk0)
  #print(prev0)
#prev0

#realization#
[11, 13] n#
[9, 11] io
[7, 9] at
[5, 7] iz
[3, 5] al
[0, 3] #re


In [ ]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.5 MB/s eta 0:00:00


In [ ]:
import tiktoken

# Initialize the Byte Pair Encoding tokenizer for GPT-2
tokenizer = tiktoken.get_encoding("gpt2")

# Define a sample text with complex and simple words
text = "unbelievable amazingness AI revolutionizing technologies"
text="ما هي هذه الأمور والتي كنا نتحدث عنها ونشاهدها"

# Encode the text using BPE (this includes both tokenization and encoding)
token_ids = tokenizer.encode(text)
print("Token IDs:", token_ids)

# Decode the token IDs back into subwords to show what BPE has done
decoded_tokens = [tokenizer.decode([token_id]) for token_id in token_ids]
print("Subwords:", decoded_tokens)

Token IDs: [25405, 12919, 18923, 229, 22654, 18923, 229, 148, 108, 29519, 28981, 148, 96, 25405, 30335, 26897, 42092, 23525, 41486, 22654, 18923, 225, 23338, 12919, 18923, 228, 41486, 148, 255, 38843, 148, 104, 17550, 117, 23338, 29519, 12919, 42092, 23338, 148, 112, 12919, 29519, 38843, 29519, 12919]
Subwords: ['م', 'ا', ' �', '�', 'ي', ' �', '�', '�', '�', 'ه', ' ال', '�', '�', 'م', 'و', 'ر', ' و', 'ال', 'ت', 'ي', ' �', '�', 'ن', 'ا', ' �', '�', 'ت', '�', '�', 'د', '�', '�', ' �', '�', 'ن', 'ه', 'ا', ' و', 'ن', '�', '�', 'ا', 'ه', 'د', 'ه', 'ا']


In [ ]:
!pip install sentencepiece

In [ ]:
import sentencepiece as spm
s = spm.SentencePieceProcessor(model_file='spm.model')
for n in range(5):
  s.encode('New York', out_type=str, enable_sampling=True, alpha=0.1, nbest_size=-1)


OSError: Not found: "spm.model": No such file or directory Error #2

In [ ]:
import re
from collections import defaultdict

class BPE:
    def __init__(self, vocab_size):
        self.vocab_size = vocab_size
        self.merges = {}
        self.vocab = {}

    def train(self, data):
        self.vocab = self._get_vocab(data)
        while len(self.vocab) < self.vocab_size:
            pairs = self._get_stats(self.vocab)
            if not pairs:
                break
            best_pair = max(pairs, key=pairs.get)
            self.merges[best_pair] = len(self.vocab)
            self.vocab = self._merge_vocab(best_pair, self.vocab)
        self._build_vocab()

    def encode(self, text):
        tokens = list(text.encode('utf-8'))
        while len(tokens) >= 2:
            stats = self._get_stats(tokens)
            pair = min(stats, key=lambda p: self.merges.get(p, float('inf')))
            if pair not in self.merges:
                break
            idx = self.merges[pair]
            tokens = self._merge(tokens, pair, idx)
        return tokens

    def decode(self, tokens):
        tokens = b"".join(self.vocab[idx] for idx in tokens)
        return tokens.decode('utf-8', errors='replace')

    def _get_vocab(self, data):
        vocab = defaultdict(int)
        for line in data:
            for word in line.split():
                vocab[' '.join(list(word)) + ' </w>'] += 1
        return vocab

    def _get_stats(self, vocab):
        pairs = defaultdict(int)
        for word, freq in vocab.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                pairs[symbols[i], symbols[i + 1]] += freq
        return pairs

    def _merge_vocab(self, pair, vocab):
        v_out = {}
        bigram = re.escape(' '.join(pair))
        p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
        for word in vocab:
            w_out = p.sub(''.join(pair), word)
            v_out[w_out] = vocab[word]
        return v_out

    def _merge(self, ids, pair, idx):
        newids = []
        i = 0
        while i < len(ids):
            if i < len(ids) - 1 and tuple(pair) == tuple(ids[i:i + 2]):
                newids.append(idx)
                i += 2
            else:
                newids.append(ids[i])
                i += 1
        return newids

    def _build_vocab(self):
         self.vocab = {i: bytes([i]) for i in range(256)}
         for pair, idx in self.merges.items():
            self.vocab[idx] = self.vocab[pair[0]] + self.vocab[pair[1]]

# Example usage:
data = ["this is the first sentence", "this is the second sentence", "and this is the third sentence"]
bpe = BPE(vocab_size=50)
bpe.train(data)

text = "this is another sentence"
encoded_text = bpe.encode(text)
decoded_text = bpe.decode(encoded_text)

print("Encoded:", encoded_text)
print("Decoded:", decoded_text)

KeyError: 't'